In [304]:
import numpy as np
import math as m

In [305]:

import opensim as osim
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import json
import os

### Import Transformed Markers

In [306]:
markers_df = pd.read_csv('model_update/tps_warping_results/markers_transformed.csv', index_col=1)

In [307]:
markers_df[markers_df['body']=='pelvis']

,body,location
name,,
ASIS_r,pelvis,[-0.00279209 -0.006827 0.1093674 ]
ASIS_l,pelvis,[ 0.00279209 0.006827 -0.1093674 ]
PSIS_r,pelvis,[-0.12988813 0.00789047 0.03996727]
PSIS_l,pelvis,[-0.12774221 0.01410826 -0.04601452]
femur_l_center_in_pelvis,pelvis,[-0.03951082 -0.07010434 -0.09239859]
femur_r_center_in_pelvis,pelvis,[-0.04875945 -0.08137492 0.07899979]
ilium_r,pelvis,[-0.0621728 0.0392263 0.11088392]
ilium_l,pelvis,[-0.0585356 0.05442049 -0.11038004]
isch_spine_r,pelvis,[-0.09318305 -0.08050016 0.04363959]


### Import Transformed Muscle Paths

In [308]:
mscles_df = pd.read_csv('model_update/tps_warping_results/muscles_transformed.csv', index_col=1)

### Import Transformed Wrapping Surfaces

In [309]:
wrp_df = pd.read_csv('model_update/tps_warping_results/wrp_transformed.csv', index_col=1)

### Parse current scaled model

In [310]:
tree=ET.parse("rajagopal_p03_1_forces_scaled.osim")
root = tree.getroot()

### Update reference geometry

In [311]:
ref_geom_dic = {'body': [], 'Mesh name' : [], 'Mesh file' : []}

ref_geom_dic['body'].append("pelvis"); ref_geom_dic['Mesh name'].append("pelvis_geom_1"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\r_pelvis.stl')
ref_geom_dic['body'].append("pelvis"); ref_geom_dic['Mesh name'].append("pelvis_geom_2"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\l_pelvis.stl')
ref_geom_dic['body'].append("pelvis"); ref_geom_dic['Mesh name'].append("pelvis_geom_3"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\sacrum.stl')

ref_geom_dic['body'].append("femur_r"); ref_geom_dic['Mesh name'].append("femur_r_geom_1"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\femur_r.stl')
ref_geom_dic['body'].append("femur_l"); ref_geom_dic['Mesh name'].append("femur_l_geom_1"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\femur_l.stl')

ref_geom_dic['body'].append("tibia_r"); ref_geom_dic['Mesh name'].append("tibia_r_geom_1"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\tibia_r.stl')
ref_geom_dic['body'].append("tibia_r"); ref_geom_dic['Mesh name'].append("tibia_r_geom_2"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\fibula_r.stl')

ref_geom_dic['body'].append("tibia_l"); ref_geom_dic['Mesh name'].append("tibia_l_geom_1"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\tibia_l.stl')
ref_geom_dic['body'].append("tibia_l"); ref_geom_dic['Mesh name'].append("tibia_l_geom_2"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\fibula_l.stl')

ref_geom_dic['body'].append("patella_r"); ref_geom_dic['Mesh name'].append("patella_r_geom_1"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\patella_r.stl')
ref_geom_dic['body'].append("patella_l"); ref_geom_dic['Mesh name'].append("patella_l_geom_1"); ref_geom_dic['Mesh file'].append('model_update\\tps_warping_results\\patella_l.stl')

In [312]:
ref_geom_df = pd.DataFrame(ref_geom_dic).set_index('Mesh name')

In [313]:
for Mesh in root.iter('Mesh'):
    point = Mesh.attrib['name']
    if point in ref_geom_df.index:
        new_text = ref_geom_df.loc[point, 'Mesh file']
        # print(point, new_text)
        Mesh.find('mesh_file').text = new_text
        Mesh.find('scale_factors').text = '1 1 1'
    else: pass

### Update muscle paths

In [314]:
for PathPoint in root.iter('PathPoint'):
    point = PathPoint.attrib['name']
    if point in mscles_df.index:
        location = mscles_df.loc[point, 'location']
        new_text = location[1:-1]
        # print(point, new_text)
        PathPoint.find('location').text = new_text
    else: pass

### Update Translation of joint centres

In [315]:
femur_r_center_in_pelvis = markers_df.loc['femur_r_center_in_pelvis', 'location']
femur_l_center_in_pelvis = markers_df.loc['femur_l_center_in_pelvis', 'location']
knee_l_center_in_femur = markers_df.loc['knee_l_center_in_femur_l', 'location']
knee_r_center_in_femur = markers_df.loc['knee_r_center_in_femur_r', 'location']
patella_l_center_in_femur = markers_df.loc['patella_l_in_femur_l', 'location']
patella_r_center_in_femur = markers_df.loc['patella_r_in_femur_r', 'location']

talus_l_center_in_tibia = markers_df.loc['talus_l_center_in_tibia', 'location']
talus_r_center_in_tibia = markers_df.loc['talus_r_center_in_tibia', 'location']

In [316]:
for joint in root.iter('CustomJoint'):
    
    name = joint.attrib['name']
    #ground_pelvis
    if joint.attrib['name'] ==  "ground_pelvis":
        for frame in joint.iter('PhysicalOffsetFrame'):
            if frame.attrib['name'] == "pelvis_offset":
                new_text = femur_r_center_in_pelvis[1:-1]
                print('old r hip translation', frame.find('translation').text)
                frame.find('translation').text = new_text
                print('new r hip translation', frame.find('translation').text)
            else: pass

    if joint.attrib['name'] ==  "hip_r":
        for frame in joint.iter('PhysicalOffsetFrame'):
            if frame.attrib['name'] == "pelvis_offset":
                new_text = femur_r_center_in_pelvis[1:-1]
                print('old r hip translation', frame.find('translation').text)
                frame.find('translation').text = new_text
                print('new r hip translation', frame.find('translation').text)
            else: pass
    
    elif joint.attrib['name'] ==  "hip_l":
        for frame in joint.iter('PhysicalOffsetFrame'):
            if frame.attrib['name'] == "pelvis_offset":
                new_text = femur_l_center_in_pelvis[1:-1]
                frame.find('translation').text = new_text
            else: pass
            
    elif joint.attrib['name'] ==  "walker_knee_r":
        for frame in joint.iter('PhysicalOffsetFrame'):
            if frame.attrib['name'] == "femur_r_offset":
                new_text = knee_r_center_in_femur[1:-1]
                print('old r walker knee translation in femur',frame.find('translation').text)
                frame.find('translation').text = new_text
                print('new r walker knee translation in femur',frame.find('translation').text)
            #if frame.attrib['name'] == "tibia_r_offset":
            #    new_text = f"{walker_knee_r_tibia_r_offset[0]}, {walker_knee_r_tibia_r_offset[1]}, {walker_knee_r_tibia_r_offset[2]}"
            #    print ('old r tibia translation', frame.find('translation').text)
            #    frame.find('translation').text = new_text
            #    print ('new r tibia translation', frame.find('translation').text)
            else: pass    
    
    elif joint.attrib['name'] ==  "walker_knee_l":
        for frame in joint.iter('PhysicalOffsetFrame'):
            if frame.attrib['name'] == "femur_l_offset":
                new_text = knee_l_center_in_femur[1:-1]
                frame.find('translation').text = new_text
    #        if frame.attrib['name'] == "tibia_l_offset":
    #            new_text = f"{walker_knee_l_tibia_l_offset[0]}, {walker_knee_l_tibia_l_offset[1]}, {walker_knee_l_tibia_l_offset[2]}"
    #            frame.find('translation').text = new_text
            else: pass

    elif joint.attrib['name'] ==  "patellofemoral_r":
       for frame in joint.iter('PhysicalOffsetFrame'):
           if frame.attrib['name'] == "femur_r_offset":
               new_text = knee_r_center_in_femur[1:-1]
               print('patellofemoral r old', frame.find('translation').text)
               frame.find('translation').text = new_text
               print('patellofemoral r new', frame.find('translation').text)
           else: pass
    
    elif joint.attrib['name'] ==  "patellofemoral_l":
       for frame in joint.iter('PhysicalOffsetFrame'):
           if frame.attrib['name'] == "femur_l_offset":
               new_text = knee_l_center_in_femur[1:-1]
               frame.find('translation').text = new_text
           else: pass

for joint in root.iter('PinJoint'):
    
    name = joint.attrib['name']

    if joint.attrib['name'] ==  "ankle_r":
       for frame in joint.iter('PhysicalOffsetFrame'):
           if frame.attrib['name'] == "tibia_r_offset":
               new_text = talus_r_center_in_tibia [1:-1]
               print('ankle r old', frame.find('translation').text)
               frame.find('translation').text = new_text
               print('ankle r new', frame.find('translation').text)
           else: pass
    
    elif joint.attrib['name'] ==  "ankle_l":
       for frame in joint.iter('PhysicalOffsetFrame'):
           if frame.attrib['name'] == "tibia_l_offset":
               new_text = talus_l_center_in_tibia [1:-1]
               frame.find('translation').text = new_text
           else: pass
    else: pass

old r hip translation 0 0 0
new r hip translation -0.04875945 -0.08137492  0.07899979
old r hip translation -0.054749578125778159 -0.067104381248515976 0.075164411223214517
new r hip translation -0.04875945 -0.08137492  0.07899979
old r walker knee translation in femur -0.0076018198061946138 -0.38334220125280033 -0.0025840549403010121
new r walker knee translation in femur  5.47117907e-16 -3.74675888e-01 -1.11910481e-16
patellofemoral r old -0.0076018198061946138 -0.38334220125280033 -0.0025840549403010121
patellofemoral r new  5.47117907e-16 -3.74675888e-01 -1.11910481e-16
ankle r old -0.0097593388562149525 -0.3903735542485981 0
ankle r new -0.00258989 -0.39920996 -0.00206685


### Update locations of muscle wrapping surfaces

In [317]:
wrp_df.head()

,body,translation,rotation,length,radius
name,,,,,
Gmax1_at_pelvis_r,pelvis,"[-0.0627, -0.1045, 0.0585]","[-0.55311, -0.11383, 0.00116]",0.1614,0.0336
Gmax2_at_pelvis_r,pelvis,"[-0.0676, -0.0897, 0.0672]","[-0.70373, -0.12605, 0.00116]",0.1030,0.0339
Gmax3_at_pelvis_r,pelvis,"[-0.0696, -0.0947, 0.0667]","[-0.13117, -0.05397, 0.00051]",0.1066,0.0339
Gmax1_at_pelvis_l,pelvis,"[-0.0563, -0.095, -0.0773]","[0.63639, 0.01766, 0.0]",0.1666,0.0344
Gmax2_at_pelvis_l,pelvis,"[-0.0604, -0.0793, -0.0833]","[0.79343, 0.04523, 5e-05]",0.1068,0.0349


In [318]:
for WrapCylinder in root.iter('WrapCylinder'):
    point = WrapCylinder.attrib['name']
    if point in wrp_df.index:
        translation = wrp_df.loc[point, 'translation']
        new_text = translation[1:-1]
        # print(point, new_text)
        WrapCylinder.find('translation').text = new_text
    else: pass

### Export transformed model

In [319]:
tree.write('p03_tps_transformed_1.osim')